In [ ]:
pip install torch-geometric

In [ ]:
pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
import torch
#torch.manual_seed(42)
from IPython.display import clear_output
torch_version = torch.__version__
print("Torch version: ", torch_version)
pytorch_version = f"torch-{torch.__version__}.html"
!pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/$pytorch_version
!pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/$pytorch_version
!pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/$pytorch_version
!pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/$pytorch_version
#!pip install torch-geometric
clear_output()
print("Done.")

In [ ]:
! nvcc --version

In [ ]:
print(torch.__version__)

In [ ]:
print(torch_geometric.__version__)

In [ ]:
import glob

import torch

import torch.nn as nn

import torch.nn.functional as F

from torch_geometric.data import Data, DataLoader

from torch_geometric.nn import GCNConv

from torch_geometric.nn import Explainer, PGExplainer

In [ ]:
import glob
#import torch
#import torch.nn as nn
#from torch_geometric.data import Data
#from torch_geometric.nn import GINConv

In [ ]:
import os
import random
import torch_geometric.nn as gnn

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip gdrive/My\ Drive/GNN_data/pt_2_files.zip

In [ ]:
!unzip gdrive/My\ Drive/GNN_data/pt_1_files.zip

In [ ]:
!unzip gdrive/My\ Drive/GNN_data/pt_files.zip

In [ ]:
# Set the directory where the '.pt' files are stored
data_dir = "pt_files"
data_dir_1 = "pt_1_files"
data_dir_2 = "pt_2_files"

In [ ]:
pt_files = os.listdir(data_dir)

In [ ]:
pt_1_files = os.listdir(data_dir_1)

In [ ]:
pt_2_files = os.listdir(data_dir_2)

In [ ]:
# Split the list of '.pt' files into training, validation, and test sets
n_train = int(len(pt_files) * 0.8)
n_val = int(len(pt_files) * 0.1)
train_pt_files, val_pt_files, test_pt_files = pt_files[:n_train], pt_files[n_train:n_train+n_val], pt_files[n_train+n_val:]

In [ ]:
# Split the list of '.pt' files into training, validation, and test sets
n_train_1 = int(len(pt_1_files) * 0.8)
n_val_1 = int(len(pt_1_files) * 0.1)
train_pt_1_files, val_pt_1_files, test_pt_1_files = pt_1_files[:n_train], pt_1_files[n_train:n_train+n_val], pt_1_files[n_train+n_val:]

In [ ]:
# Split the list of '_2.pt' files into training, validation, and test sets
n_train_2 = int(len(pt_2_files) * 0.8)
n_val_2 = int(len(pt_2_files) * 0.1)
train_pt_2_files, val_pt_2_files, test_pt_2_files = pt_2_files[:n_train_2], pt_2_files[n_train_2:n_train_2+n_val_2], pt_2_files[n_train_2+n_val_2:]

In [ ]:
# Combine the training data sets from '.pt' and '_2.pt' files
train_data_files = train_pt_files + train_pt_1_files + train_pt_2_files

In [ ]:
val_data_files = val_pt_files + val_pt_1_files + val_pt_2_files

In [ ]:
test_data_files = test_pt_files + test_pt_1_files + test_pt_2_files

In [ ]:
from torch_geometric.data import DataLoader

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, file_paths):
        self.data = [torch.load(file_path) for file_path in file_paths]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
file_paths = []
for file_name in train_data_files:
    file_path = os.path.join(data_dir, file_name)
    if not os.path.exists(file_path):
        file_path = os.path.join(data_dir_1, file_name)
    if not os.path.exists(file_path):
        file_path = os.path.join(data_dir_2, file_name)
    if os.path.exists(file_path):
        file_paths.append(file_path)


# Create an instance of your dataset with the full paths
dataset_train = MyDataset(file_paths)

# Create a DataLoader from the dataset
train_dataloader = DataLoader(dataset_train, batch_size=64, shuffle=True)

In [ ]:
data = next(iter(val_dataloader))
print(f"x shape: {data.x.shape}")
print(f"edge_index shape: {data.edge_index.shape}")
print(f"y shape: {data.y.shape}")


In [ ]:
file_paths = []
for file_name in val_data_files:
    file_path = os.path.join(data_dir, file_name)
    if not os.path.exists(file_path):
        file_path = os.path.join(data_dir_1, file_name)
    if not os.path.exists(file_path):
        file_path = os.path.join(data_dir_2, file_name)
    if os.path.exists(file_path):
        file_paths.append(file_path)


# Create an instance of your dataset with the full paths
dataset_val = MyDataset(file_paths)

# Create a DataLoader from the dataset
val_dataloader = DataLoader(dataset_val, batch_size=64, shuffle=True)

In [ ]:
file_paths = []
for file_name in test_data_files:
    file_path = os.path.join(data_dir, file_name)
    if not os.path.exists(file_path):
        file_path = os.path.join(data_dir_1, file_name)
    if not os.path.exists(file_path):
        file_path = os.path.join(data_dir_2, file_name)
    if os.path.exists(file_path):
        file_paths.append(file_path)


# Create an instance of your dataset with the full paths
dataset_test = MyDataset(file_paths)

# Create a DataLoader from the dataset
test_dataloader = DataLoader(dataset_test, batch_size=1, shuffle=True)

In [ ]:
device = torch.device("cuda")

In [ ]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import torch.nn.functional as F
import torch.nn as nn

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(42)
        self.conv1 = GCNConv(input_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 3)

    def forward(self, x, edge_index, batch, edge_weight=None):
        print(edge_index.shape)
        print(edge_index)
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_weight)
        x = F.relu(x)
        x = self.conv3(x, edge_index, edge_weight)
        x = gnn.global_mean_pool(x, batch)
        x = self.lin(x)
        return F.log_softmax(x, dim=-1)

# Define criterion and optimizer
model = GCN(input_dim = 3, hidden_channels = 256)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
model = model.to(device)
model = model.double()
# Train and evaluate the model
for epoch in range(num_epochs):
    model.train()
    correct = 0
    for data in train_dataloader:
        optimizer.zero_grad()
        out = model(data.x.to(device), data.edge_index.to(device).long(), data.batch.to(device))
        loss = criterion(out, data.y.to(device))
        loss.backward()
        optimizer.step()
        pred = out.argmax(dim=1)
        correct += int((pred == data.y.to(device)).sum())
        #print(correct)
    train_acc = correct / len(train_dataloader.dataset)
    model.eval()
    correct = 0
    for data in val_dataloader:
        out = model(data.x.to(device), data.edge_index.to(device).long(), data.batch.to(device))
        pred = out.argmax(dim=1)
        correct += int((pred == data.y.to(device)).sum())
    val_acc = correct / len(val_dataloader.dataset)
    print(f'Epoch: {epoch}, Train Accuracy: {train_acc:.4f}, Validation Accuracy: {val_acc:.4f}')


In [ ]:
model = GCN(input_dim = 3, hidden_channels = 256)
#model.load_state_dict(torch.load('model1.pth'))
model.to(device)
#model.load_state_dict(torch.load('model1.pth'))
#model.eval()

In [ ]:
model.load_state_dict(torch.load('model1.pth', map_location=device))
model.to(device)

In [ ]:
from torch_geometric.explain import Explainer, PGExplainer
explainer = Explainer(
    model=model,
    algorithm=PGExplainer(epochs=10, lr=0.003).to(device),
    explanation_type='phenomenon',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='graph',
        return_type='raw',
    ),
    # Include only the top 10 most important edges:
    threshold_config=dict(threshold_type='topk', value=5),
)
# PGExplainer needs to be trained separately since it is a parametric
# explainer i.e it uses a neural network to generate explanations:
for epoch in range(10):
    for data in test_dataloader:
        # Move the data to the chosen device
        data = data.to(device)
        x = data.x
        x = x.to(device)

        edge_index = data.edge_index
        edge_index = edge_index.to(device)

        target = data.y
        target = target.to(device)

        batch = data.batch
        batch = batch.to(device)


        # Pass the data to the model's train method
        loss = explainer.algorithm.train(epoch, model, x, edge_index, target=target, batch=batch)
# Generate the explanation for a particular graph:
explanation = explainer(dataset[0].x, dataset[0].edge_index)
print(explanation.edge_mask)

In [ ]:
pip install captum

In [ ]:
def get_first_graph(dataloader):
    for batch in dataloader:
        graph = batch[0]
        return graph

In [ ]:
graph = get_first_graph(test_dataloader)

In [ ]:
y = graph.y
print(y)

In [ ]:
def get_second_graph(dataloader):
    count = 0
    for batch in dataloader:
        count += 1
        if count == 9:
            graph = batch[0]
            return graph


In [ ]:
graph2 = get_second_graph(test_dataloader)
y = graph2.y
print(y)

In [ ]:
from captum.attr import Saliency, IntegratedGradients

def model_forward(edge_mask, data):
    batch = torch.zeros(data.x.shape[0], dtype=int).to(device)
    out = model(data.x, data.edge_index, batch, edge_mask)
    return out


def explain(method, data, target=0):
    input_mask = torch.ones(data.edge_index.shape[1]).requires_grad_(True).to(device)
    if method == 'ig':
        ig = IntegratedGradients(model_forward)
        mask = ig.attribute(input_mask, target=target,
                            additional_forward_args=(data,),
                            internal_batch_size=data.edge_index.shape[1])
    elif method == 'saliency':
        saliency = Saliency(model_forward)
        mask = saliency.attribute(input_mask, target=target,
                                  additional_forward_args=(data,))
    else:
        raise Exception('Unknown explanation method')

    edge_mask = np.abs(mask.cpu().detach().numpy())
    if edge_mask.max() > 0:  # avoid division by zero
        edge_mask = edge_mask / edge_mask.max()
    return edge_mask

In [ ]:
import random
from collections import defaultdict
import numpy as np

def aggregate_edge_directions(edge_mask, data):
    edge_mask_dict = defaultdict(float)
    for val, u, v in list(zip(edge_mask, *data.edge_index)):
        u, v = u.item(), v.item()
        if u > v:
            u, v = v, u
        edge_mask_dict[(u, v)] += val
    return edge_mask_dict


for title, method in [('Integrated Gradients', 'ig'), ('Saliency', 'saliency')]:
    edge_mask = explain(method, graph.to(device), target=0)
    edge_mask_dict = aggregate_edge_directions(edge_mask, graph.to(device))

In [ ]:
for title, method in [('Integrated Gradients', 'ig'), ('Saliency', 'saliency')]:
    edge_mask = explain(method, graph2.to(device), target=0)
    edge_mask_dict = aggregate_edge_directions(edge_mask, graph2.to(device))
    print(f"Method: {title}")
    print("Edge\tScore")
    for edge, score in sorted(edge_mask_dict.items(), key=lambda x: x[1], reverse=True)[:20]:
        print(f"{edge}\t{score}")
    print()